In [31]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd

dataset = load_dataset('ag_news')
train_ds = dataset['train'].shuffle(seed=42).select([i for i in list(range(5000))])
validation_test_ds = dataset['test'].shuffle(seed=42).select([i for i in list(range(1000))])


display(train_ds)
train_df = pd.DataFrame(train_ds)
display(train_df)


Using custom data configuration default
Reusing dataset ag_news (C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 401.02it/s]
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548\cache-dd0ff9596fea92b0.arrow
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548\cache-12f3c4e4bf422cce.arrow


Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})

,text,label
0,Bangladesh paralysed by strikes Opposition act...,0
1,Desiring Stability Redskins coach Joe Gibbs ex...,1
2,Will Putin #39;s Power Play Make Russia Safer?...,0
3,U2 pitches for Apple New iTunes ads airing dur...,3
4,S African TV in beheading blunder Public broad...,0
...,...,...
4995,Nigerian Protection Force Leaves for Darfur (R...,0
4996,"Lawrence Dallaglio factfile 1995: In October, ...",1
4997,African leaders reject foreign intervention in...,0
4998,Ex-Troops Fill Haiti's Security Vacuum Nation ...,0


In [32]:
#wordnet part
import nltk
#nltk.download('omw-1.4')
from nltk.corpus import wordnet


from senticnet.senticnet import SenticNet
import pandas as pd

import spacy
from spacy import displacy
from lemminflect import getInflection

In [33]:
#creating antonyms list using wordnet
def antonyms_list_creation(target_word):
    antonyms_list = []
    try:
        for ann in wordnet.synsets(target_word):
            for lm in ann.lemmas():
                if lm.antonyms():
                    antonyms_list.append(lm.antonyms()[0].name())
        #print (set(antonyms_list))
        antonyms_list = list(set(antonyms_list))
        #transform _ to space
        antonyms_list = [s.replace('_', ' ') for s in antonyms_list]


        antonyms_list
        antonyms_list = [s for s in antonyms_list if ' ' not in s]
        
    except Exception as e:
        pass
    
    return antonyms_list

In [34]:
#senticnet part
senticnet_df = pd.read_excel(r'C:\Users\DELL\Text_Augmentation\senticnet\senticnet.xlsx',header=0)

In [35]:
#transform _ to space

senticnet_df['CONCEPT'] = senticnet_df['CONCEPT'].str.replace('_', ' ')


senticnet_df.set_index('CONCEPT',inplace=True)

print(senticnet_df)

               INTROSPECTION  TEMPER  ATTITUDE  SENSITIVITY PRIMARY EMOTION  \
CONCEPT                                                                       
abandon               -0.165     0.0     0.000        0.000     #melancholy   
abase                  0.000     0.0    -0.890        0.000       #loathing   
abash                  0.000     0.0    -0.341       -0.442           #fear   
abashment              0.000     0.0    -0.329        0.000        #dislike   
abate                  0.000     0.0    -0.339       -0.551           #fear   
...                      ...     ...       ...          ...             ...   
zotob worm             0.000     0.0     0.000       -0.958         #terror   
zotob wrestle          0.000     0.0     0.000       -0.962         #terror   
zotob wriggle          0.000     0.0     0.000       -0.918         #terror   
zotob writhe           0.000     0.0     0.000       -0.979         #terror   
zz                     0.000     0.0    -0.182      

In [36]:
#creating opposite words list using senticnet
def opposite_polarity_words_list_creation(target_word):
    if target_word in senticnet_df.index:
        
        INTROSPECTION = -senticnet_df.loc[target_word, 'INTROSPECTION']
        TEMPER = -senticnet_df.loc[target_word, 'TEMPER']
        ATTITUDE = -senticnet_df.loc[target_word, 'ATTITUDE']
        SENSITIVITY = -senticnet_df.loc[target_word, 'SENSITIVITY']





        extracted_df = senticnet_df[(senticnet_df['INTROSPECTION'] == INTROSPECTION) & 
                                   (senticnet_df['TEMPER'] == TEMPER) & (senticnet_df['ATTITUDE'] == ATTITUDE) & 
                                    (senticnet_df['SENSITIVITY'] == SENSITIVITY)]


        opposite_polarity_words_list = extracted_df.index.values.tolist()
        opposite_polarity_words_list = [s for s in opposite_polarity_words_list if ' ' not in s]
    
    else:
        opposite_polarity_words_list = []
    
    return opposite_polarity_words_list

In [37]:
import itertools
from textattack.transformations import WordSwap
class Antipode_WordSwap(WordSwap):
    
    
    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        antonyms_list = antonyms_list_creation(word)
        antipode_list = []
        try:
            antipode_list.append(opposite_polarity_words_list_creation(word))
        except Exception as e:
            pass
        
        antipode_list.append(antonyms_list)
        antipode_list = list(itertools.chain.from_iterable(antipode_list))
        antipode_list = list(set(antipode_list))

        return antipode_list


In [38]:
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification
from textattack.constraints.grammaticality.part_of_speech import PartOfSpeech
from textattack.constraints.semantics.bert_score import BERTScore
from textattack.augmentation import Augmenter


transformation = Antipode_WordSwap()

# Set up constraints
constraints = [RepeatModification(), StopwordModification(),
               PartOfSpeech(tagger_type='stanza', tagset='universal', allow_verb_noun_swap=True, compare_against_original=True, language_nltk='eng', language_stanza='en')
                ]
# Create augmenter with specified parameters
augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=0.1, transformations_per_example=1)

Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

Use device: gpu
Loading: tokenize
Loading: pos
Done loading processors!


In [39]:
# Augment!
# additional parameters can be modified if not during initiation




def results_list_creation(s):

    try:
        results = augmenter.augment(s)
        
    
    except Exception as e:
        results = ['Error occured in text generation .']
    
    return results

import time
from multiprocessing import Pool
from concurrent import futures

time_sta = time.perf_counter()
results_list = []

#print(list(train_df['text']))

with futures.ThreadPoolExecutor() as executor:
    results_list_itera = executor.map(results_list_creation, list(train_df['text'])) 

for results in results_list_itera:
    results_list.append(results)

time_end = time.perf_counter()
tim = time_end - time_sta

In [40]:
print(tim/3600)




1.0678513047777778


In [41]:
display(results_list)

[['Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a start, the day after 18 people keycode in explosions at a political rally.'],
 ['Desiring Stability Redskins coach Joe Gibbs expects few minor personnel changes in the offseason and wants to instill a culture of inconstancy in Washington.'],
 ['disinherit Putin #39;s powerlessness Play Make Russia Safer? Outwardly, Russia has not changed since the barrage of terrorist defend that culminated in the school massacre in Beslan on Sept.'],
 ['U2 pitches for Apple New iTunes ads airing during baseball games Tuesday will lack the advertising-shy Irish rockers.'],
 ['S African TV in beheading blunder private broadcaster SABC apologises after news bulletin disprove footage of American beheaded in Iraq.'],
 ['A Cosmic Storm: When Galaxy Clusters Collide Astronomers have lost what they are calling the imperfect cosmic chymical, a galaxy cluster pile-up so powerful its energy output is sec

In [42]:
import itertools

original_sentences_list = []

for i in range(len(train_df)):
    number_of_generated_sentences_per_original_sentence = len(results_list[i])
    original_sentences = [list(train_df['text'])[i] for j in range(number_of_generated_sentences_per_original_sentence)]
    original_sentences_list.append(original_sentences)
display(original_sentences_list)

original_sentences_list_1d = list(itertools.chain.from_iterable(original_sentences_list))
results_list_1d = list(itertools.chain.from_iterable(results_list))


[['Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally.'],
 ['Desiring Stability Redskins coach Joe Gibbs expects few major personnel changes in the offseason and wants to instill a culture of stability in Washington.'],
 ['Will Putin #39;s Power Play Make Russia Safer? Outwardly, Russia has not changed since the barrage of terrorist attacks that culminated in the school massacre in Beslan on Sept.'],
 ['U2 pitches for Apple New iTunes ads airing during baseball games Tuesday will feature the advertising-shy Irish rockers.'],
 ['S African TV in beheading blunder Public broadcaster SABC apologises after news bulletin shows footage of American beheaded in Iraq.'],
 ['A Cosmic Storm: When Galaxy Clusters Collide Astronomers have found what they are calling the perfect cosmic storm, a galaxy cluster pile-up so powerful its energy output is second only to the Big Bang

In [43]:
import pandas as pd 
augmented_data = {'original_text':original_sentences_list_1d,'perturbed_text':results_list_1d}
augmented_data_df = pd.DataFrame(augmented_data)
display(augmented_data_df)


,original_text,perturbed_text
0,Bangladesh paralysed by strikes Opposition act...,Bangladesh paralysed by strikes Opposition act...
1,Desiring Stability Redskins coach Joe Gibbs ex...,Desiring Stability Redskins coach Joe Gibbs ex...
2,Will Putin #39;s Power Play Make Russia Safer?...,disinherit Putin #39;s powerlessness Play Make...
3,U2 pitches for Apple New iTunes ads airing dur...,U2 pitches for Apple New iTunes ads airing dur...
4,S African TV in beheading blunder Public broad...,S African TV in beheading blunder private broa...
...,...,...
4995,Nigerian Protection Force Leaves for Darfur (R...,Nigerian Protection Force Leaves for Darfur (R...
4996,"Lawrence Dallaglio factfile 1995: In October, ...","Lawrence Dallaglio factfile 1995: In October, ..."
4997,African leaders reject foreign intervention in...,African leaders reject foreign nonintervention...
4998,Ex-Troops Fill Haiti's Security Vacuum Nation ...,Ex-Troops Fill Haiti's Security Vacuum Nation ...


In [44]:
#remove augmented_data_df['original_text'] == augmented_data_df['perturbed_text']
target = augmented_data_df.index[augmented_data_df['original_text'] == augmented_data_df['perturbed_text']]
augmented_data_df = augmented_data_df.drop(target)
augmented_data_df = augmented_data_df.reset_index(drop=True)
display(augmented_data_df)

,original_text,perturbed_text
0,Bangladesh paralysed by strikes Opposition act...,Bangladesh paralysed by strikes Opposition act...
1,Desiring Stability Redskins coach Joe Gibbs ex...,Desiring Stability Redskins coach Joe Gibbs ex...
2,Will Putin #39;s Power Play Make Russia Safer?...,disinherit Putin #39;s powerlessness Play Make...
3,U2 pitches for Apple New iTunes ads airing dur...,U2 pitches for Apple New iTunes ads airing dur...
4,S African TV in beheading blunder Public broad...,S African TV in beheading blunder private broa...
...,...,...
4978,Nigerian Protection Force Leaves for Darfur (R...,Nigerian Protection Force Leaves for Darfur (R...
4979,"Lawrence Dallaglio factfile 1995: In October, ...","Lawrence Dallaglio factfile 1995: In October, ..."
4980,African leaders reject foreign intervention in...,African leaders reject foreign nonintervention...
4981,Ex-Troops Fill Haiti's Security Vacuum Nation ...,Ex-Troops Fill Haiti's Security Vacuum Nation ...


In [45]:
#save to csv file
augmented_data_df.to_csv( r'C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\augmented_datasets\Antipode\with_pos\allow_verb_noun_swap=True\run3\dataset.csv')